In [1]:
# init
from os import path
import numpy as np
from pynput import keyboard
from pedalboard import Pedalboard, Chorus, Reverb
from pedalboard.io import AudioFile
from keyboard_layout import KeyboardLayout
from tone import Tone

smplRate = 44100

IS_LOOPED = False
timeline_duration = 2
timeline = np.zeros(timeline_duration * smplRate, dtype = np.float32)


board = Pedalboard([Chorus(), Reverb(room_size=0.25)])

keyboard_layout = KeyboardLayout()
keyboard_layout.load('complete left to right')



complete left to right loaded as keyboard layout successfully.


In [4]:
tonemaker_1 = Tone(harmonics = [(2,2),(2.4,2.2),(2.7,2.6),(3.3,3),(4,4)])
def scaling_1(point):
    return 40 * 2**(point/12)
def scaling_2(point):
    scale = [2,2,1,2,2,2,1]
    result = 1
    for i in range(point):
        result += scale[i%len(scale)]
    return result

In [8]:
# open audio
import pyaudio

position = 0

def callback(in_data, frame_count, time_info, status):
    # If len(data) is less than requested frame_count, PyAudio automatically
    # assumes the stream is finished, and the stream stops.
    global position
    if timeline.size >= (position + frame_count):
        data = timeline[position:position+frame_count].copy()
        if not IS_LOOPED:
            timeline[position:position+frame_count] = 0
        position += frame_count
    else:
        data = timeline[position:].copy()
        if not IS_LOOPED:
            timeline[position:] = 0
        position += frame_count
        position -= timeline.size
        data = np.append(data, timeline[:position])
        if not IS_LOOPED:
            timeline[:position] = 0
            
    data = board.process(data,smplRate,reset = False)
    return (data, pyaudio.paContinue)

paud = pyaudio.PyAudio()
stream = paud.open(
            format=pyaudio.paFloat32,
            channels=1,
            rate=smplRate,
            output=True,
            stream_callback=callback
            )



# listen to keyboard
streams = []
def on_press(key):
    try:
        fundamental = scaling_1(scaling_2(keyboard_layout.value.index(key.char)))
    except AttributeError:
        # print('special key {0} pressed'.format(
        #     key))
        return
    global position
    tone = tonemaker_1.make(fundamental)/10
    data_size = tone.size
    if timeline.size >= (position + data_size):
        timeline[position:position+data_size] += tone
    else:
        tone.size - timeline.size
        timeline[position:] += tone[:timeline.size-position]
        timeline[:position+tone.size-timeline.size] += tone[timeline.size-position:]
    
def on_release(key):
    if key == keyboard.Key.esc:
        # Stop listener
        stream.close()
        paud.terminate()
        print("stoped.")
        return False

# Collect events until released
# with keyboard.Listener(
#         on_press=on_press,
#         on_release=on_release) as listener:
#     listener.join()

# ...or, in a non-blocking fashion:
listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release)
listener.start()
print("started...")

started...


This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


stoped.


In [6]:
# closing audio in case of emergency
stream.close()
paud.terminate()

In [7]:
# clear timline
timeline = np.zeros(timeline_duration * smplRate, dtype = np.float32)

In [11]:
#save timeline
new_file_name = input('enter your new timeline name')
if not new_file_name:
    print("didn't save.")
else:
    file_address = path.join('database/saved_sounds/' , new_file_name + '.npy')
    with open(file_address,'wb') as file:
        np.save(file,timeline)
        print(new_file_name + " saved as timeline successfully.")


didn't save.
